<a href="https://colab.research.google.com/github/Najamzfr/ML-Zoomcamp-Project2/blob/main/MLzoomcamp_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import minimize

In [ ]:
# Step 1: Create directory structure
def create_directories():
    directories = [
        "data/raw",
        "data/processed",
        "notebooks",
        "src",
        "models/saved_models",
        "models/model_analysis",
        "reports/figures",
        "tests",
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

In [ ]:
# Step 2: Download stock data
def download_stock_data(tickers, start_date, end_date):
    raw_data_path = "data/raw"
    for ticker in tickers:
        print(f"Downloading data for {ticker}...")
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        file_path = os.path.join(raw_data_path, f"{ticker}.csv")
        stock_data.to_csv(file_path)
        print(f"Saved {ticker} data to {file_path}")

In [ ]:
data = pd.read_csv("data/raw/AAPL.csv")
data = data.rename(columns={'Price': 'Date'})
data = data.iloc[2:]
data

,Date,Close,High,Low,Open,Volume
2,2018-01-02,40.524349212646484,40.53376126592413,39.818595981979136,40.03032410496728,102223600
3,2018-01-03,40.51728439331055,41.06306851269611,40.453769198969376,40.587860421113284,118071600
4,2018-01-04,40.705482482910156,40.80899350294978,40.48199473854038,40.59020819429411,89738400
5,2018-01-05,41.16893768310547,41.2559794312343,40.710198809708736,40.80194658438808,94640000
6,2018-01-08,41.016029357910156,41.3124444417875,40.91722074030425,41.016029357910156,82271200
...,...,...,...,...,...,...
1757,2024-12-23,255.27000427246094,255.64999389648438,253.4499969482422,254.77000427246094,40858800
1758,2024-12-24,258.20001220703125,258.2099914550781,255.2899932861328,255.49000549316406,23234700
1759,2024-12-26,259.0199890136719,260.1000061035156,257.6300048828125,258.19000244140625,27237100
1760,2024-12-27,255.58999633789062,258.70001220703125,253.05999755859375,257.8299865722656,42355300


In [ ]:
df = pd.read_csv("data/raw/TSLA.csv", header=2, parse_dates=["Date"], index_col="Date")
# rename unnamed columns with these Close	High	Low	Open	Volume
df = df.rename(columns={'Unnamed: 1': 'Close','Unnamed: 2': 'High','Unnamed: 3': 'Low','Unnamed: 4': 'Open','Unnamed: 5': 'Volume'})
df

,Close,High,Low,Open,Volume
Date,,,,,
2018-01-02,21.368668,21.474001,20.733334,20.799999,65283000
2018-01-03,21.150000,21.683332,21.036667,21.400000,67822500
2018-01-04,20.974667,21.236668,20.378668,20.858000,149194500
2018-01-05,21.105333,21.149332,20.799999,21.108000,68868000
2018-01-08,22.427334,22.468000,21.033333,21.066668,147891000
...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,431.000000,72698100
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400


In [ ]:
# Step 3: Data Preprocessing
def preprocess_data():
    raw_data_path = "data/raw"
    processed_data_path = "data/processed"

    for file in os.listdir(raw_data_path):
        if file.endswith(".csv"):
            print(f"Processing {file}...")
            file_path = os.path.join(raw_data_path, file)
            data = pd.read_csv(file_path, header=2, parse_dates=["Date"], index_col="Date")  # Start from row 3
            data = df.rename(columns={'Unnamed: 1': 'Close',
                                      'Unnamed: 2': 'High',
                                      'Unnamed: 3': 'Low',
                                      'Unnamed: 4': 'Open',
                                      'Unnamed: 5': 'Volume'})

            # Retain only relevant columns
            data = data[["Close"]]

            # Remove NaN rows (if any)
            data = data.dropna()

            # Save the processed data
            processed_file_path = os.path.join(processed_data_path, file)
            data.to_csv(processed_file_path)
            print(f"Saved processed data to {processed_file_path}")


In [ ]:
# Step 4: Feature Engineering
def feature_engineering():
    processed_data_path = "data/processed"
    for file in os.listdir(processed_data_path):
        if file.endswith(".csv"):
            print(f"Engineering features for {file}...")
            file_path = os.path.join(processed_data_path, file)
            data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")

            # Generate lag features
            data["Lag_1"] = data["Close"].shift(1)
            data["Lag_2"] = data["Close"].shift(2)

            # Calculate moving averages
            data["MA_5"] = data["Close"].rolling(window=5).mean()
            data["MA_10"] = data["Close"].rolling(window=10).mean()

            # Drop rows with NaN values introduced by lagging/rolling
            data = data.dropna()

            # Save engineered data
            feature_file_path = os.path.join(processed_data_path, f"features_{file}")
            data.to_csv(feature_file_path)
            print(f"Saved feature-engineered data to {feature_file_path}")

In [ ]:
# Step 5: Model Training
def train_models():
    processed_data_path = "data/processed"
    models_path = "models/saved_models"
    analysis_path = "models/model_analysis"
    figures_path = "reports/figures"

    os.makedirs(models_path, exist_ok=True)
    os.makedirs(analysis_path, exist_ok=True)
    os.makedirs(figures_path, exist_ok=True)

    predictions = {}

    for file in os.listdir(processed_data_path):
        if file.startswith("features_") and file.endswith(".csv"):
            print(f"Training models for {file}...")
            file_path = os.path.join(processed_data_path, file)
            data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")

            # Prepare data for Linear Regression
            X = data[["Lag_1", "Lag_2", "MA_5", "MA_10"]]
            y = data["Close"]

            # Split into training and testing sets
            train_size = int(len(data) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Linear Regression Model
            lr_model = LinearRegression()
            lr_model.fit(X_train, y_train)
            lr_predictions = lr_model.predict(X_test)

            # Save Linear Regression Model
            lr_model_path = os.path.join(models_path, f"linear_regression_{file}.pkl")
            with open(lr_model_path, "wb") as f:
                pickle.dump(lr_model, f)
            print(f"Saved Linear Regression model to {lr_model_path}")

            # Evaluate Linear Regression Model
            lr_mse = mean_squared_error(y_test, lr_predictions)
            lr_rmse = np.sqrt(lr_mse)
            print(f"Linear Regression RMSE for {file}: {lr_rmse}")

            # Align predictions by index (dates)
            predictions[file] = pd.Series(lr_predictions, index=y_test.index)

    return predictions

In [ ]:
# Step 6: Portfolio Optimization
def optimize_portfolio(predictions):
    print("Optimizing portfolio...")

    # Combine all predicted returns into a DataFrame
    pred_returns = pd.DataFrame(predictions)
    mean_returns = pred_returns.mean()  # Expected returns
    cov_matrix = pred_returns.cov()  # Covariance matrix

    # Objective function: minimize negative Sharpe ratio
    def neg_sharpe(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    # Constraints: weights sum to 1
    constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    # Bounds: weights between 0 and 1
    bounds = tuple((0, 1) for _ in range(len(mean_returns)))

    # Initial guess
    initial_weights = [1 / len(mean_returns)] * len(mean_returns)

    # Optimization
    result = minimize(neg_sharpe, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)

    optimal_weights = result.x
    print("Optimal portfolio weights:", optimal_weights)

    # Plot efficient frontier
    plt.figure(figsize=(10, 6))
    plt.scatter(pred_returns.std(), mean_returns, label="Individual Stocks")
    plt.scatter(np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))),
                np.dot(optimal_weights, mean_returns), c="red", label="Optimal Portfolio", marker="X", s=100)
    plt.title("Efficient Frontier")
    plt.xlabel("Risk (Standard Deviation)")
    plt.ylabel("Return")
    plt.legend()
    plt.grid()
    plt.savefig("reports/figures/efficient_frontier.png")
    plt.close()

In [ ]:
# Step 7: Backtest Portfolio
def backtest_portfolio(optimal_weights, pred_returns, historical_prices):
    print("Backtesting the optimized portfolio...")

    # Combine historical returns using optimal weights
    portfolio_returns = (pred_returns @ optimal_weights)
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    # Performance metrics
    annualized_return = portfolio_returns.mean() * 252  # Assuming 252 trading days
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility

    # Print performance metrics
    print(f"Annualized Return: {annualized_return:.2%}")
    print(f"Annualized Volatility: {annualized_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    # Plot cumulative returns
    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_returns, label="Optimized Portfolio")
    plt.title("Backtest: Portfolio Cumulative Returns")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")
    plt.legend()
    plt.grid()
    plt.savefig("reports/figures/portfolio_backtest.png")
    plt.close()


In [ ]:
# Step 8: Sensitivity analysis
def sensitivity_analysis(mean_returns, cov_matrix):
    print("Performing sensitivity analysis...")
    risk_aversion_levels = [0.1, 0.5, 1, 2, 5]  # Example risk aversion values
    results = []

    for risk_aversion in risk_aversion_levels:
        # Adjusted Sharpe ratio objective (accounting for risk aversion)
        def neg_sharpe_adjusted(weights):
            portfolio_return = np.dot(weights, mean_returns)
            portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            return -(portfolio_return - risk_aversion * portfolio_volatility)

        # Optimization
        result = minimize(neg_sharpe_adjusted, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)
        results.append((risk_aversion, result.x))

    # Display results
    for level, weights in results:
        print(f"Risk Aversion {level}: {weights}")


In [ ]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import minimize


# Step 1: Create directory structure
def create_directories():
    directories = [
        "data/raw",
        "data/processed",
        "notebooks",
        "src",
        "models/saved_models",
        "models/model_analysis",
        "reports/figures",
        "tests",
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Step 2: Download stock data
def download_stock_data(tickers, start_date, end_date):
    raw_data_path = "data/raw"
    for ticker in tickers:
        print(f"Downloading data for {ticker}...")
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        file_path = os.path.join(raw_data_path, f"{ticker}.csv")
        stock_data.to_csv(file_path)
        print(f"Saved {ticker} data to {file_path}")

# Step 3: Data Preprocessing
def preprocess_data():
    raw_data_path = "data/raw"
    processed_data_path = "data/processed"

    for file in os.listdir(raw_data_path):
        if file.endswith(".csv"):
            print(f"Processing {file}...")
            file_path = os.path.join(raw_data_path, file)
            data = pd.read_csv(file_path, header=2, parse_dates=["Date"], index_col="Date")
            data = data.rename(columns={'Unnamed: 1': 'Close',
                                        'Unnamed: 2': 'High',
                                        'Unnamed: 3': 'Low',
                                        'Unnamed: 4': 'Open',
                                        'Unnamed: 5': 'Volume'})

            # Retain only relevant columns
            data = data[["Close"]]

            # Remove NaN rows (if any)
            data = data.dropna()

            # Save the processed data
            processed_file_path = os.path.join(processed_data_path, file)
            data.to_csv(processed_file_path)
            print(f"Saved processed data to {processed_file_path}")


# Step 4: Feature Engineering
def feature_engineering():
    processed_data_path = "data/processed"
    for file in os.listdir(processed_data_path):
        if file.endswith(".csv"):
            print(f"Engineering features for {file}...")
            file_path = os.path.join(processed_data_path, file)
            data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")

            # Generate lag features
            data["Lag_1"] = data["Close"].shift(1)
            data["Lag_2"] = data["Close"].shift(2)

            # Calculate moving averages
            data["MA_5"] = data["Close"].rolling(window=5).mean()
            data["MA_10"] = data["Close"].rolling(window=10).mean()

            # Drop rows with NaN values introduced by lagging/rolling
            data = data.dropna()

            # Save engineered data
            feature_file_path = os.path.join(processed_data_path, f"features_{file}")
            data.to_csv(feature_file_path)
            print(f"Saved feature-engineered data to {feature_file_path}")

# Step 5: Model Training
def train_models():
    processed_data_path = "data/processed"
    models_path = "models/saved_models"
    analysis_path = "models/model_analysis"
    figures_path = "reports/figures"

    os.makedirs(models_path, exist_ok=True)
    os.makedirs(analysis_path, exist_ok=True)
    os.makedirs(figures_path, exist_ok=True)

    predictions = {}

    for file in os.listdir(processed_data_path):
        if file.startswith("features_") and file.endswith(".csv"):
            print(f"Training models for {file}...")
            file_path = os.path.join(processed_data_path, file)
            data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")

            # Prepare data for Linear Regression
            X = data[["Lag_1", "Lag_2", "MA_5", "MA_10"]]
            y = data["Close"]

            # Split into training and testing sets
            train_size = int(len(data) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]

            # Linear Regression Model
            lr_model = LinearRegression()
            lr_model.fit(X_train, y_train)
            lr_predictions = lr_model.predict(X_test)

            # Save Linear Regression Model
            lr_model_path = os.path.join(models_path, f"linear_regression_{file}.pkl")
            with open(lr_model_path, "wb") as f:
                pickle.dump(lr_model, f)
            print(f"Saved Linear Regression model to {lr_model_path}")

            # Evaluate Linear Regression Model
            lr_mse = mean_squared_error(y_test, lr_predictions)
            lr_rmse = np.sqrt(lr_mse)
            print(f"Linear Regression RMSE for {file}: {lr_rmse}")

            # Align predictions by index (dates)
            predictions[file] = pd.Series(lr_predictions, index=y_test.index)

    return predictions

# Step 6: Model Performance Evaluation
def evaluate_model(y_test, predictions, model_name):
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print(f"--- {model_name} Evaluation ---")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")

    # Residual plot
    residuals = y_test - predictions
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, residuals, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.title(f"{model_name} Residuals")
    plt.xlabel("Actual Values")
    plt.ylabel("Residuals")
    plt.grid()
    plt.savefig(f"reports/figures/{model_name}_residuals.png")
    plt.close()


# Step 7: Portfolio Optimization
def optimize_portfolio(predictions):
    print("Optimizing portfolio...")

    # Combine all predicted returns into a DataFrame
    pred_returns = pd.DataFrame(predictions)
    mean_returns = pred_returns.mean()  # Expected returns
    cov_matrix = pred_returns.cov()  # Covariance matrix

    # Objective function: minimize negative Sharpe ratio
    def neg_sharpe(weights):
        portfolio_return = np.dot(weights, mean_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = portfolio_return / portfolio_volatility
        return -sharpe_ratio

    # Constraints: weights sum to 1
    constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    # Bounds: weights between 0 and 1
    bounds = tuple((0, 1) for _ in range(len(mean_returns)))

    # Initial guess
    initial_weights = [1 / len(mean_returns)] * len(mean_returns)

    # Optimization
    result = minimize(neg_sharpe, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)

    optimal_weights = result.x
    print("Optimal portfolio weights:", optimal_weights)

    # Plot efficient frontier
    plt.figure(figsize=(10, 6))
    plt.scatter(pred_returns.std(), mean_returns, label="Individual Stocks")
    plt.scatter(np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))),
                np.dot(optimal_weights, mean_returns), c="red", label="Optimal Portfolio", marker="X", s=100)
    plt.title("Efficient Frontier")
    plt.xlabel("Risk (Standard Deviation)")
    plt.ylabel("Return")
    plt.legend()
    plt.grid()
    plt.savefig("reports/figures/efficient_frontier.png")
    plt.close()

# Step 8: Portfolio Performance Evaluation
def evaluate_portfolio(portfolio_returns):
    # Annualized metrics
    annualized_return = portfolio_returns.mean() * 252  # 252 trading days
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility
    max_drawdown = (portfolio_returns.cummin() - portfolio_returns.cummax()).min()

    print("Portfolio Evaluation:")
    print(f"Annualized Return: {annualized_return:.2%}")
    print(f"Annualized Volatility: {annualized_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Maximum Drawdown: {max_drawdown:.2%}")

    # Plot cumulative returns
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1
    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_returns, label="Optimized Portfolio")
    plt.title("Portfolio Cumulative Returns")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")
    plt.legend()
    plt.grid()
    plt.savefig("reports/figures/portfolio_cumulative_returns.png")
    plt.close()

# Step 9: Backtest Portfolio
def backtest_portfolio(optimal_weights, pred_returns, historical_prices):
    print("Backtesting the optimized portfolio...")

    # Combine historical returns using optimal weights
    portfolio_returns = (pred_returns @ optimal_weights)
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    # Performance metrics
    annualized_return = portfolio_returns.mean() * 252  # Assuming 252 trading days
    annualized_volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_volatility

    # Print performance metrics
    print(f"Annualized Return: {annualized_return:.2%}")
    print(f"Annualized Volatility: {annualized_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

    # Plot cumulative returns
    plt.figure(figsize=(10, 6))
    plt.plot(cumulative_returns, label="Optimized Portfolio")
    plt.title("Backtest: Portfolio Cumulative Returns")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")
    plt.legend()
    plt.grid()
    plt.savefig("reports/figures/portfolio_backtest.png")
    plt.close()

# Step 10: Sensitivity analysis
def sensitivity_analysis(mean_returns, cov_matrix):
    print("Performing sensitivity analysis...")
    risk_aversion_levels = [0.1, 0.5, 1, 2, 5]  # Example risk aversion values
    results = []

    for risk_aversion in risk_aversion_levels:
        # Adjusted Sharpe ratio objective (accounting for risk aversion)
        def neg_sharpe_adjusted(weights):
            portfolio_return = np.dot(weights, mean_returns)
            portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
            return -(portfolio_return - risk_aversion * portfolio_volatility)

        # Optimization
        result = minimize(neg_sharpe_adjusted, initial_weights, method="SLSQP", bounds=bounds, constraints=constraints)
        results.append((risk_aversion, result.x))

    # Display results
    for level, weights in results:
        print(f"Risk Aversion {level}: {weights}")


def main():
    # Step 1: Create necessary directories
    print("Step 1: Creating directories...")
    create_directories()

    # Step 2: Define stock tickers and date range
    print("Step 2: Downloading stock data...")
    tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN"]  # Example stock symbols
    start_date = "2018-01-01"
    end_date = "2024-12-31"
    download_stock_data(tickers, start_date, end_date)

    # Step 3: Preprocess downloaded data
    print("Step 3: Preprocessing data...")
    preprocess_data()

    # Step 4: Perform feature engineering
    print("Step 4: Engineering features...")
    feature_engineering()

    # Step 5: Train models and generate predictions
    print("Step 5: Training models...")
    predictions = train_models()

    # Step 6: Optimize portfolio based on predictions
    print("Step 6: Optimizing portfolio...")
    optimize_portfolio(predictions)


if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Step 1: Creating directories...
Step 2: Downloading stock data...
Saved AAPL data to data/raw/AAPL.csv
Saved MSFT data to data/raw/MSFT.csv



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Saved TSLA data to data/raw/TSLA.csv
Saved GOOGL data to data/raw/GOOGL.csv
Saved AMZN data to data/raw/AMZN.csv
Step 3: Preprocessing data...
Processing AMZN.csv...
Saved processed data to data/processed/AMZN.csv
Processing MSFT.csv...
Saved processed data to data/processed/MSFT.csv
Processing AAPL.csv...
Saved processed data to data/processed/AAPL.csv
Processing GOOGL.csv...
Saved processed data to data/processed/GOOGL.csv
Processing TSLA.csv...
Saved processed data to data/processed/TSLA.csv
Step 4: Engineering features...
Engineering features for features_features_features_MSFT.csv...
Saved feature-engineered data to data/processed/features_features_features_features_MSFT.csv
Engineering features for features_features_features_AAPL.csv...
Saved feature-engineered data to data/processed/features_features_features_features_AAPL.csv
Engineering features for features_features_features_TSLA.csv...
Saved feature-engineered data to data/processed/features_features_features_features_TSLA.c

In [ ]:
!zip -r reports.zip reports/

from google.colab import files
files.download('reports.zip')


  adding: reports/ (stored 0%)
  adding: reports/figures/ (stored 0%)
  adding: reports/figures/arima_predictions_features_AMZN.csv.png (deflated 6%)
  adding: reports/figures/lr_predictions_features_AAPL.csv.png (deflated 6%)
  adding: reports/figures/lr_predictions_features_features_AMZN.csv.png (deflated 6%)
  adding: reports/figures/lr_predictions_features_features_features_GOOGL.csv.png (deflated 7%)
  adding: reports/figures/arima_predictions_features_features_features_features_AMZN.csv.png (deflated 7%)
  adding: reports/figures/arima_predictions_features_TSLA.csv.png (deflated 7%)
  adding: reports/figures/arima_predictions_features_features_features_MSFT.csv.png (deflated 6%)
  adding: reports/figures/arima_predictions_features_GOOGL.csv.png (deflated 5%)
  adding: reports/figures/arima_predictions_features_features_features_features_TSLA.csv.png (deflated 7%)
  adding: reports/figures/lr_predictions_features_features_features_TSLA.csv.png (deflated 7%)
  adding: reports/figur

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>